# Autoencoder

In [6]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from    PIL import Image
from    matplotlib import pyplot as plt

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype(np.float32) / 255., x_test.astype(np.float32) / 255.
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
# image grid
new_im = Image.new('L', (280, 280))

image_size = 28*28
h_dim = 20
num_epochs = 5
batch_size = 100
learning_rate = 1e-3

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [7]:
class AE(tf.keras.Model):

    def __init__(self):
        super(AE, self).__init__()
        # 784 => 512
        self.fc1 = keras.layers.Dense(512)
        # 512 => h
        self.fc2 = keras.layers.Dense(h_dim)    # bottleneck 
        # h => 512
        self.fc3 = keras.layers.Dense(512)
        # 512 => image
        self.fc4 = keras.layers.Dense(image_size)

    def encode(self, x):
        x = tf.nn.relu(self.fc1(x))
        x = (self.fc2(x))
        return x

    def decode_logits(self, h):
        x = tf.nn.relu(self.fc3(h))
        x = self.fc4(x)
        return x

    def decode(self, h):
        return tf.nn.sigmoid(self.decode_logits(h))

    def call(self, inputs, training=None, mask=None):
        # encoder
        h = self.encode(inputs)
        # decode
        x_reconstructed_logits = self.decode_logits(h)
        return x_reconstructed_logits

In [8]:
model = AE()
model.build(input_shape=(4, image_size))
model.summary()
optimizer = keras.optimizers.Adam(learning_rate)

Model: "ae_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  401920    
_________________________________________________________________
dense_5 (Dense)              multiple                  10260     
_________________________________________________________________
dense_6 (Dense)              multiple                  10752     
_________________________________________________________________
dense_7 (Dense)              multiple                  402192    
Total params: 825,124
Trainable params: 825,124
Non-trainable params: 0
_________________________________________________________________


In [9]:
dataset = tf.data.Dataset.from_tensor_slices(x_train) # here doesn't need labels
dataset = dataset.shuffle(batch_size * 5).batch(batch_size)

num_batches = x_train.shape[0] // batch_size

for epoch in range(num_epochs):

    for step, x in enumerate(dataset):

        x = tf.reshape(x, [-1, image_size])

        with tf.GradientTape() as tape:

            # Forward pass
            x_reconstruction_logits = model(x)

            # Compute reconstruction loss and kl divergence
            # Scaled by `image_size` for each individual pixel.
            reconstruction_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=x_reconstruction_logits)
            reconstruction_loss = tf.reduce_sum(reconstruction_loss) / batch_size

        gradients = tape.gradient(reconstruction_loss, model.trainable_variables) 
        gradients,_ = tf.clip_by_global_norm(gradients, 15)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if (step + 1) % 50 == 0:
            print("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}"
                  .format(epoch + 1, num_epochs, step + 1, num_batches, float(reconstruction_loss)))

     # Save the reconstructed images of last batch
    out_logits = model(x[:batch_size // 2])
    out = tf.nn.sigmoid(out_logits)  # out is just the logits, use sigmoid
    out = tf.reshape(out, [-1, 28, 28]).numpy() * 255

    x = tf.reshape(x[:batch_size // 2], [-1, 28, 28])

    x_concat = tf.concat([x, out], axis=0).numpy() * 255.
    x_concat = x_concat.astype(np.uint8)

    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = x_concat[index]
            im = Image.fromarray(im, mode='L')
            new_im.paste(im, (i, j))
            index += 1

    new_im.save('images/vae_reconstructed_epoch_%d.png' % (epoch + 1))
    plt.imshow(np.asarray(new_im))
    plt.show()
    print('New images saved !')

W1009 17:31:49.680379 4550337984 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_core/python/data/util/random_seed.py:58: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch[1/5], Step [50/600], Reconst Loss: 160.8146
Epoch[1/5], Step [100/600], Reconst Loss: 127.1601
Epoch[1/5], Step [150/600], Reconst Loss: 109.0554
Epoch[1/5], Step [200/600], Reconst Loss: 103.8838
Epoch[1/5], Step [250/600], Reconst Loss: 85.9174
Epoch[1/5], Step [300/600], Reconst Loss: 85.5601
Epoch[1/5], Step [350/600], Reconst Loss: 89.2355
Epoch[1/5], Step [400/600], Reconst Loss: 80.7757
Epoch[1/5], Step [450/600], Reconst Loss: 81.8119
Epoch[1/5], Step [500/600], Reconst Loss: 87.8791
Epoch[1/5], Step [550/600], Reconst Loss: 79.7579
Epoch[1/5], Step [600/600], Reconst Loss: 79.2986


FileNotFoundError: [Errno 2] No such file or directory: 'images/vae_reconstructed_epoch_1.png'